In [2]:
import sys

import polars as pl
import torch
from torch.optim.lr_scheduler import CosineAnnealingLR

from data_loader.TimeSeriesModule import MultiPartDataModule
from model_runner.train.titanl_train import TitanTrain
from models.PatchMixer.common.configs import PatchMixerConfigMonthly
from models.PatchTST.common.configs import PatchTSTConfigMonthly
from models.Titan.Titans import LMMModel, LMMSeq2SeqModel
import torch.nn as nn

from models.Titan.common.configs import TitanConfigMonthly

'''
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
https://developer.nvidia.com/cuda-12-8-0-download-archive
'''

MAC_DIR = '../data/'
WINDOW_DIR = 'C:/Users/USER/PycharmProjects/research/data/'

if sys.platform == 'win32':
    DIR = WINDOW_DIR
    print(torch.cuda.is_available())
    print(torch.cuda.device_count())
    print(torch.version.cuda)
    print(torch.__version__)
    print(torch.cuda.get_device_name(0))
    print(torch.__version__)
else:
    DIR = MAC_DIR

True
1
12.8
2.9.0.dev20250716+cu128
NVIDIA GeForce RTX 5080
2.9.0.dev20250716+cu128


In [3]:

target_dyn_demand_monthly = pl.read_parquet(DIR + 'target_dyn_demand_monthly.parquet')

titan_cfg = TitanConfigMonthly()
mixer_cfg = PatchMixerConfigMonthly()
tst_cfg = PatchTSTConfigMonthly()

data_module = MultiPartDataModule(
    df = target_dyn_demand_monthly,
    config = titan_cfg,
    batch_size = 128,
    val_ratio = 0.2,
    is_running = False
)

plan_yyyymm = 202401
train_loader = data_module.get_train_loader()
val_loader = data_module.get_val_loader()
anchor_loader = data_module.get_inference_loader_at_plan(
    plan_dt = plan_yyyymm,
    parts_filter = None,
    fill_missing = 'ffill'
)

from utils.validation_utils import collect_indices

fp_tr = collect_indices(train_loader)
fp_va = collect_indices(val_loader)
print("fp shapes:", fp_tr.shape, fp_va.shape)
print("approx equal:", torch.allclose(fp_tr.mean(0), fp_va.mean(0)))

fp shapes: torch.Size([87, 2]) torch.Size([22, 2])
approx equal: False


In [ ]:
from torch.optim import AdamW

base_lmm_model = LMMModel(titan_cfg)
lmm_seq2seq_model = LMMSeq2SeqModel(titan_cfg)

trainer = TitanTrain()

models = {
    'lmm': base_lmm_model,
    'lmm_seq2seq': lmm_seq2seq_model,
}

trained = {}
histories = {}

for name, mdl in models.items():
    optimizer = AdamW(mdl.parameters(), lr=1e-3, weight_decay = 1e-4)
    scheduler = CosineAnnealingLR(optimizer, T_max = 10)

    print(f"\n=== Training: {name} ===")
    trained_model, (tr_loss, val_loss) = trainer.train_model_with_tta(
        model = mdl,
        train_loader = train_loader,
        val_loader = val_loader
    )

    trained[name] = trained_model
    histories[name] = (tr_loss, val_loss)

    torch.save(trained_model.state_dict(), DIR + f'fit/titan_{name}.pt')
    print(f"Saved: fit/titan_{name}.pt")

